In [47]:
import json
from selenium import webdriver
import pandas as pd
import requests
import xml.etree.ElementTree as ET

In [48]:
driver = webdriver.Chrome('C:\\Users\\Joanna\\AppData\\Local\\Programs\\Python\\Python39\\venv\\Scripts\\chromedriver.exe')
keywords = ['czerwona sukienka']
driver.get('https://www.google.pl/search?q='+keywords[0])

In [49]:
webelements = driver.find_elements_by_xpath('//div[@class="rc"]/div/a')
urls = []
for elem in webelements:
    urls.append(elem.get_attribute('href'))
print(urls)

['https://domodi.pl/czerwone-sukienki', 'https://allegro.pl/listing?string=czerwona%20sukienka', 'https://allani.pl/wyszukaj/sukienki/czerwone', 'https://www.zalando.pl/odziez-damska-sukienki/_czerwony/', 'https://www.renee.pl/sukienki/czerwone', 'https://ohso.pl/c4558,4614,0,0,,czerwone-sukienki', 'https://modnakiecka.pl/pl/c/Czerwone-Sukienki/2240726', 'https://www.c-and-a.com/pl/pl/shop/trend-czerwona-sukienka', 'https://mosquito-sklep.pl/pl/238-sukienki-czerwone']


In [50]:
headings1 = []
titles = []
bodies = []
for url in urls[:1]:
    driver.get(url)
    heading1 = driver.find_elements_by_tag_name('h1')
    headings1.append(heading1[0].text)
    title = driver.title
    titles.append(title)
    body = driver.find_elements_by_tag_name('body')
    bodies.append(body[0].text)

In [51]:
df = pd.DataFrame(
    {'site' : urls[:1],
     'title': titles,
     'h1' : headings1,
     'content' : bodies
     })
df.head()

,site,title,h1,content
0,https://domodi.pl/czerwone-sukienki,"Czerwone sukienki, zima 2020 w Domodi","Czerwone sukienki, zima 2020",Skompletuj swoją szafę na nowy rok z rabatami ...


In [52]:
category = 'performance'
for url in urls[:1]:
    api_content = requests.get('https://pagespeedonline.googleapis.com/pagespeedonline/v5/runPagespeed?url={}&category={}'.format(url, category)).content
    content = json.loads(api_content)

In [53]:
for key, value in content['lighthouseResult']['categories'][category].items():
    print('key: {}, value: {}'.format(key,value))

key: id, value: performance
key: title, value: Performance
key: score, value: 0.92
key: auditRefs, value: [{'id': 'first-contentful-paint', 'weight': 15, 'group': 'metrics'}, {'id': 'speed-index', 'weight': 15, 'group': 'metrics'}, {'id': 'largest-contentful-paint', 'weight': 25, 'group': 'metrics'}, {'id': 'interactive', 'weight': 15, 'group': 'metrics'}, {'id': 'total-blocking-time', 'weight': 25, 'group': 'metrics'}, {'id': 'cumulative-layout-shift', 'weight': 5, 'group': 'metrics'}, {'id': 'first-cpu-idle', 'weight': 0}, {'id': 'max-potential-fid', 'weight': 0}, {'id': 'first-meaningful-paint', 'weight': 0}, {'id': 'estimated-input-latency', 'weight': 0}, {'id': 'render-blocking-resources', 'weight': 0, 'group': 'load-opportunities'}, {'id': 'uses-responsive-images', 'weight': 0, 'group': 'load-opportunities'}, {'id': 'offscreen-images', 'weight': 0, 'group': 'load-opportunities'}, {'id': 'unminified-css', 'weight': 0, 'group': 'load-opportunities'}, {'id': 'unminified-javascript',

In [54]:
ids = []
for elem in content['lighthouseResult']['categories'][category]['auditRefs']:
    print(elem['id'])
    print('weight: {}'. format(elem['weight']))
#     ids.append(elem['id'])
print(ids)
len(ids)

first-contentful-paint
weight: 15
speed-index
weight: 15
largest-contentful-paint
weight: 25
interactive
weight: 15
total-blocking-time
weight: 25
cumulative-layout-shift
weight: 5
first-cpu-idle
weight: 0
max-potential-fid
weight: 0
first-meaningful-paint
weight: 0
estimated-input-latency
weight: 0
render-blocking-resources
weight: 0
uses-responsive-images
weight: 0
offscreen-images
weight: 0
unminified-css
weight: 0
unminified-javascript
weight: 0
unused-css-rules
weight: 0
unused-javascript
weight: 0
uses-optimized-images
weight: 0
uses-webp-images
weight: 0
uses-text-compression
weight: 0
uses-rel-preconnect
weight: 0
server-response-time
weight: 0
redirects
weight: 0
uses-rel-preload
weight: 0
efficient-animated-content
weight: 0
duplicated-javascript
weight: 0
legacy-javascript
weight: 0
total-byte-weight
weight: 0
uses-long-cache-ttl
weight: 0
dom-size
weight: 0
critical-request-chains
weight: 0
user-timings
weight: 0
bootup-time
weight: 0
mainthread-work-breakdown
weight: 0
fon

0

In [55]:
i=0
for key, value in content['lighthouseResult']['audits'].items():
    if key in ids:
        print('key: {},\n\tid: {}, \n\tscore: {},\n\ttitle: {}, \n\tdescription: {}'.format(key,value['id'], value['score'], value['title'], value['description']))